In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from isanlp.processor_sentence_splitter import ProcessorSentenceSplitter
from isanlp.en.processor_tokenizer_nltk_en import ProcessorTokenizerNltkEn
from isanlp import PipelineCommon
from isanlp_srl_allennlp.processor_srl_allennlp import ProcessorSrlAllennlp


model_path = 'https://s3-us-west-2.amazonaws.com/allennlp/models/bert-base-srl-2019.06.17.tar.gz'

tokenizer = ProcessorTokenizerNltkEn()
splitter = ProcessorSentenceSplitter()
srl_proc = ProcessorSrlAllennlp(model_path)

ppl = PipelineCommon([
    (tokenizer, ['text'], {0 : 'tokens'}),
    (splitter, ['tokens'], {0 : 'sentences'}),
    (srl_proc, ['tokens', 'sentences'], {0 : 'srl'})
])

In [3]:
#results = ppl('Jack has built a house')
results = ppl('Did Uriah honestly think he could beat the game in under three hours? Mama washed the frame.')

In [5]:
results

{'text': 'Did Uriah honestly think he could beat the game in under three hours? Mama washed the frame.',
 'tokens': [<isanlp.annotation.Token at 0x7fbd8d3246d8>,
 'sentences': [<isanlp.annotation.Sentence at 0x7fbe2968bda0>,
 'srl': [[<isanlp.annotation.Event at 0x7fbe29666d30>,
  [<isanlp.annotation.Event at 0x7fbd93c82ba8>]]}

In [11]:
from isanlp.annotation_repr import CSentence

def print_event(event, tokens):
    print('Verb: ', ' '.join(tokens[e].text for e in range(event.pred[0], event.pred[1])))
    for arg in event.args:
        print('Arg: ', arg.tag, ' '.join(tokens[e].text for e in range(arg.begin, arg.end)))
        

for sent_num, srl_sent in enumerate(results['srl']):
    for event in srl_sent:
        print_event(event, CSentence(results['tokens'], results['sentences'][sent_num]))

Verb:  Did
Verb:  think
Arg:  ARG0 Uriah
Arg:  ARGM honestly
Arg:  ARG1 he could beat the game in under three hours
Verb:  could
Verb:  beat
Arg:  ARG0 he
Arg:  ARGM could
Arg:  ARG1 the game
Arg:  ARGM in under three hours
Verb:  washed
Arg:  ARG0 Mama
Arg:  ARG1 the frame


In [13]:
from isanlp.processor_remote import ProcessorRemote

proc = ProcessorRemote('192.168.1.69', 3333, 'default')
proc('My data was washing a car. My mom was washing the frame.')

{'text': 'My data was washing a car. My mom was washing the frame.',
 'tokens': [<isanlp.annotation.Token at 0x7fbd8cc61c88>,
 'sentences': [<isanlp.annotation.Sentence at 0x7fbd8cc61ef0>,
 'srl': [[<isanlp.annotation.Event at 0x7fbd8cc61fd0>,
   <isanlp.annotation.Event at 0x7fbd8cc55198>]]}